In [24]:
from pathlib import Path

import ezdxf
import pandas as pd
from ezdxf.entities import MText
from shapely import LineString, Polygon, Point

from src.processing.splitting_numbers import split_number

In [2]:
NUMBERS_LAYERS = ["номера"]
LINES_LAYERS = ["полосы"]
CONTOURS_LAYERS = ["контуры"]
MIN_DISTANCE = 0.01

numbers_layers = NUMBERS_LAYERS
lines_layers = LINES_LAYERS
contours_layers = CONTOURS_LAYERS
min_distance = MIN_DISTANCE

In [3]:
doc = ezdxf.readfile(Path("data/example_topographic_plan.dxf"))

In [4]:
# Сбор номеров и их позиций
numbers_data = []

# Собираем отдельно TEXT и MTEXT
for layer in numbers_layers:
    text_entities = doc.modelspace().query(f'TEXT[layer=="{layer}"]')
    mtext_entities = doc.modelspace().query(f'MTEXT[layer=="{layer}"]')

    for text in list(text_entities) + list(mtext_entities):
        number = text.plain_text().replace('\n', ' ') if isinstance(text, MText) else text.plain_text()
        numbers_data.append({
            'number': number,
            'position': Point(text.dxf.insert[0], text.dxf.insert[1])
        })

numbers_df = pd.DataFrame(numbers_data)
numbers_df

,number,position
0,1,POINT (3075.965582309878 1696.605175781559)
1,2,POINT (3074.823165523758 1693.90068742274)
2,3,POINT (3108.838205298518 1707.285109871409)
3,4,POINT (3114.150988844972 1707.46955728739)
4,5,POINT (3115.690642463274 1707.58678205736)
5,6,POINT (3115.100445342571 1706.18062074892)
6,7,POINT (3114.699609216242 1705.090515570424)
7,10,POINT (3104.043722283615 1698.543651078497)
8,11,POINT (3105.090899215862 1695.948120853597)
9,12,POINT (3106.332887793541 1693.701322612177)


In [5]:
# Сбор геометрических фигур
shapes_data = []

# Сбор линий
for layer in lines_layers:
    lines = doc.modelspace().query(f'LINE[layer=="{layer}"]')
    polylines = doc.modelspace().query(f'LWPOLYLINE[layer=="{layer}"]')

    for line in lines:
        shape = LineString([(line.dxf.start.x, line.dxf.start.y),
                            (line.dxf.end.x, line.dxf.end.y)])
        shapes_data.append({
            'geometry': shape,
            'type': 'LineString'
        })

    for pline in polylines:
        shape = LineString([(float(x), float(y)) for x, y in list(pline.vertices())])
        shapes_data.append({
            'geometry': shape,
            'type': 'LineString'
        })

# Сбор контуров
for layer in contours_layers:
    contours = doc.modelspace().query(f'LWPOLYLINE[layer=="{layer}"]')
    for contour in contours:
        shape = Polygon([(float(x), float(y)) for x, y in list(contour.vertices())])
        shapes_data.append({
            'geometry': shape,
            'type': 'Polygon'
        })

shapes_df = pd.DataFrame(shapes_data)
shapes_df

,geometry,type
0,LINESTRING (3092.268998306507 1693.40414713145...,LineString
1,LINESTRING (3099.275403871401 1706.81442616184...,LineString
2,"POLYGON ((3106.916523995033 1690.249262830753,...",Polygon
3,"POLYGON ((3075.045526933311 1693.259229309259,...",Polygon
4,"POLYGON ((3106.496601561342 1689.562929311654,...",Polygon


In [12]:
# Связывание номеров и фигур
numbers_shapes_data = []
if not numbers_df.empty and not shapes_df.empty:
    # Создаем геометрические точки для всех номеров
    number_points = numbers_df.apply(lambda row: row['position'], axis=1)

    # Для каждой фигуры проверяем все номера
    for shape_id, shape_row in shapes_df.iterrows():
        shape_type = shape_row['type']
        shape_geom = shape_row['geometry']

        # Вычисляем расстояния до всех точек
        if shape_type == 'LineString':
            distances = number_points.apply(lambda p: shape_geom.distance(p))
        else:
            distances = number_points.apply(lambda p: p.distance(shape_geom.exterior))

        # Находим номера, которые находятся достаточно близко
        close_numbers = distances[distances < min_distance]

        for number_id in close_numbers.index:
            numbers_shapes_data.append({
                'number_id': number_id,
                'shape_id': shape_id
            })

numbers_shapes_df = pd.DataFrame(numbers_shapes_data)
numbers_shapes_df

,number_id,shape_id
0,14,0
1,15,1
2,12,2
3,13,2
4,0,3
5,1,3
6,16,4


In [13]:
# Сбор деревьев (точечных объектов)
unassigned_numbers = numbers_df.index.difference(numbers_shapes_df['number_id'])

trees_data = []
for number_id in unassigned_numbers:
    trees_data.append({
        'number_id': number_id
    })

trees_df = pd.DataFrame(trees_data)
trees_df

,number_id
0,2
1,3
2,4
3,5
4,6
5,7
6,8
7,9
8,10
9,11


In [46]:
topographic_plan_data = []

for number_id in trees_df['number_id']:
    for _split_number in split_number(numbers_df.iloc[number_id]['number']):
        topographic_plan_data.append(
            {
                'origin_number': numbers_df.iloc[number_id]['number'],
                'number_position': numbers_df.iloc[number_id]['position'],
                'split_number': _split_number,
                'type': 'Point',
                'geometry': numbers_df.iloc[number_id]['position'],
                'size': None
            }
        )

for numbers_shapes_id in numbers_shapes_df.index:
    
    number_id = numbers_shapes_df.iloc[numbers_shapes_id]['number_id']
    shape_id = numbers_shapes_df.iloc[numbers_shapes_id]['shape_id']
    shape: LineString | Polygon = shapes_df.iloc[shape_id]['geometry']
    shape_type = shapes_df.iloc[shape_id]['type']
    
    for _split_number in split_number(numbers_df.iloc[number_id]['number']):
        topographic_plan_data.append(
            {
                'origin_number': numbers_df.iloc[number_id]['number'],
                'number_position': numbers_df.iloc[number_id]['position'],
                'split_number': _split_number,
                'type': shape_type,
                'geometry': shape,
                'size': shape.length if isinstance(shape, LineString) else shape.area
            }
        )

topographic_plan = pd.DataFrame(topographic_plan_data)
topographic_plan

,origin_number,number_position,split_number,type,geometry,size
0,3,POINT (3108.838205298518 1707.285109871409),3,Point,POINT (3108.838205298518 1707.285109871409),NaN
1,4,POINT (3114.150988844972 1707.46955728739),4,Point,POINT (3114.150988844972 1707.46955728739),NaN
2,5,POINT (3115.690642463274 1707.58678205736),5,Point,POINT (3115.690642463274 1707.58678205736),NaN
3,6,POINT (3115.100445342571 1706.18062074892),6,Point,POINT (3115.100445342571 1706.18062074892),NaN
4,7,POINT (3114.699609216242 1705.090515570424),7,Point,POINT (3114.699609216242 1705.090515570424),NaN
5,10,POINT (3104.043722283615 1698.543651078497),10,Point,POINT (3104.043722283615 1698.543651078497),NaN
6,11,POINT (3105.090899215862 1695.948120853597),11,Point,POINT (3105.090899215862 1695.948120853597),NaN
7,12,POINT (3106.332887793541 1693.701322612177),12,Point,POINT (3106.332887793541 1693.701322612177),NaN
8,14,POINT (3104.361115387819 1692.888423763281),14,Point,POINT (3104.361115387819 1692.888423763281),NaN
9,13,POINT (3107.488870740065 1689.420932022136),13,Point,POINT (3107.488870740065 1689.420932022136),NaN
